In [30]:
import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz 


from sklearn import tree
from pandas import read_table, DataFrame
from os import system

In [31]:
train_file = pd.read_csv('train.csv')
test_file = pd.read_csv('test.csv')


In [32]:
len(train_file)

26729

In [33]:
len(test_file)

11456

In [34]:
#Name and OutcomeSubtype are deleted
train_file.drop(["Name", "OutcomeSubtype"], axis=1, inplace=True)
test_file.drop(["Name"], axis=1, inplace=True)

In [35]:
def calc_age_in_years(x):
    x=str(x)
    if x == 'nan': return 0
    age = int(x.split()[0])
    if x.find('year')> -1: return age
    if x.find('month')> -1: return age/12.
    if x.find('week')> -1: return age/52.
    if x.find('day')> -1: return age/365.
    else: return 0

In [36]:
train_file['AgeInYears'] = train_file.AgeuponOutcome.apply(calc_age_in_years)
test_file['AgeInYears'] = test_file.AgeuponOutcome.apply(calc_age_in_years)

In [37]:
#Separating IDs
train_id = train_file[["AnimalID"]]
test_id = test_file[["ID"]]
train_file.drop(["AnimalID"], axis=1, inplace=True)
test_file.drop(["ID"], axis=1, inplace=True)

In [38]:
#Separating target variable
train_outcome = train_file["OutcomeType"]
train_file.drop(["OutcomeType"], axis=1, inplace=True)

In [39]:
len(train_file)

26729

In [40]:
len(test_file)

11456

30548.0

In [69]:
categorical_vars = ['AnimalType', 'SexuponOutcome', 'AgeuponOutcome']

In [75]:
#Encode the categorical data, with the complete set (train and test)

train_file["train"] = 1
test_file["train"] = 0
combined = pd.concat([train_file, test_file])
#sex_dummies2 = pd.get_dummies(df_test['Sex'])
combined_encoded = pd.get_dummies(combined, columns = categorical_vars)
combined_encoded.drop(["Color"], axis=1, inplace=True)
combined_encoded.drop(["Breed"], axis=1, inplace=True)
combined_encoded.drop(["DateTime"], axis=1, inplace=True)
                                

#train = combined_encoded[combined_encoded["train"] == 1]
#test = combined_encoded[combined_encoded["train"] == 0]
#train.drop(["train_0","train_1"], axis=1, inplace=True)
#test.drop(["train_0","train_1"], axis=1, inplace=True)

In [76]:
combined_encoded.head()

,AgeInYears,train,AnimalType_Cat,AnimalType_Dog,SexuponOutcome_Intact Female,SexuponOutcome_Intact Male,SexuponOutcome_Neutered Male,SexuponOutcome_Spayed Female,SexuponOutcome_Unknown,AgeuponOutcome_0 years,...,AgeuponOutcome_5 years,AgeuponOutcome_6 days,AgeuponOutcome_6 months,AgeuponOutcome_6 years,AgeuponOutcome_7 months,AgeuponOutcome_7 years,AgeuponOutcome_8 months,AgeuponOutcome_8 years,AgeuponOutcome_9 months,AgeuponOutcome_9 years
0,1.000000,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.000000,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.057692,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.000000,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
combined.head()

,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,AgeInYears,train
0,2/12/2014 18:22,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,1.000000,1
1,10/13/2013 12:44,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,1.000000,1
2,1/31/2015 12:28,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,2.000000,1
3,7/11/2014 19:09,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,0.057692,1
4,11/15/2013 12:52,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,2.000000,1


In [77]:
#boolean indexing to separate train vs test
train = combined_encoded[combined_encoded["train"] == 1]
test = combined_encoded[combined_encoded["train"] == 0]
train.drop(["train"], axis=1, inplace=True)
test.drop(["train"], axis=1, inplace=True)

C:\Users\julia.dills\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\julia.dills\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split



In [79]:
#Training a RF to get some metrics
X_train, X_val, y_train, y_val = train_test_split(train, train_outcome, test_size=0.2)
forest = RandomForestClassifier(n_estimators=250, n_jobs=2)
forest.fit(X_train, y_train)
y_pred_val = forest.predict(X_val)

In [112]:
y_pred_train = forest.predict(X_train)

In [115]:
y_pred_train

array(['Adoption', 'Transfer', 'Transfer', ..., 'Adoption', 'Transfer',
       'Transfer'], dtype=object)

In [83]:
X_train.shape, y_train.shape

((21383, 53), (21383,))

In [87]:
X_val.shape, y_val.shape

((5346, 53), (5346,))

In [ ]:
#21383 + 5346 = len of training data set but NOT with test included
#allows to verify the split

In [91]:
#did not have to encode the prediction
y_pred_val

array(['Return_to_owner', 'Adoption', 'Adoption', ..., 'Return_to_owner',
       'Return_to_owner', 'Transfer'], dtype=object)

In [92]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_val, y_pred_val))
print(accuracy_score(y_val, y_pred_val))

             precision    recall  f1-score   support

   Adoption       0.66      0.80      0.72      2146
       Died       0.00      0.00      0.00        47
 Euthanasia       0.35      0.07      0.12       326
Return_to_owner       0.45      0.43      0.44       964
   Transfer       0.70      0.65      0.68      1863

avg / total       0.61      0.63      0.61      5346

0.630751964085


In [101]:
from sklearn.cross_validation import cross_val_score

In [103]:
logit = RandomForestClassifier()
cross_val_score(logit, X_train, y_train, 'accuracy', cv=10)

array([ 0.64065421,  0.64345794,  0.62365591,  0.62628625,  0.62675398,
        0.63985033,  0.6272217 ,  0.62581852,  0.63610851,  0.63529963])

In [ ]:
#model works evenly over the whole data set

In [116]:
#Training a RF with the complete training set
forest = RandomForestClassifier(n_estimators=500, n_jobs=2)
forest.fit(train, train_outcome)
y_pred = forest.predict_proba(test)
y_pred_train_prob = forest.predict_proba(train)

In [94]:
y_pred

array([[ 0.        ,  0.        ,  0.        ,  0.10138417,  0.89861583],
       [ 0.53720195,  0.00189884,  0.02563255,  0.26175041,  0.17351626],
       [ 0.39868413,  0.        ,  0.02081146,  0.09913627,  0.48136814],
       ..., 
       [ 0.00300401,  0.00583395,  0.08931354,  0.00907241,  0.8927761 ],
       [ 0.31441063,  0.        ,  0.05631625,  0.43208207,  0.19719106],
       [ 0.01560971,  0.        ,  0.21646038,  0.46018034,  0.30774957]])

In [110]:
y_pred.shape

(11456, 5)

In [121]:
#want to figure out this bc that is how kaggle is checking
#from sklearn.metrics import log_loss

In [118]:
y_pred_train.shape, y_pred_train_prob.shape

((21383,), (26729, 5))

In [119]:
y_pred_train

array(['Adoption', 'Transfer', 'Transfer', ..., 'Adoption', 'Transfer',
       'Transfer'], dtype=object)

In [120]:
y_pred_train_prob

array([[ 0.55398103,  0.00162869,  0.02028074,  0.22828054,  0.19582899],
       [ 0.46649718,  0.        ,  0.0220611 ,  0.11236457,  0.39907715],
       [ 0.44310705,  0.00145976,  0.03880017,  0.31269183,  0.20394119],
       ..., 
       [ 0.36467406,  0.00229618,  0.06754941,  0.40114523,  0.16433512],
       [ 0.        ,  0.06383536,  0.0722514 ,  0.        ,  0.86391324],
       [ 0.00557791,  0.00578231,  0.16798112,  0.0111115 ,  0.80954717]])

In [122]:
#log_loss(y_pred_train, y_pred_train_prob)

In [96]:
results = pd.read_csv("sample_submission.csv")



In [97]:
results.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0


In [98]:
results['Adoption'], results['Died'], results['Euthanasia'], results['Return_to_owner'], results['Transfer'] = y_pred[:,0], y_pred[:,1], y_pred[:,2], y_pred[:,3], y_pred[:,4]


In [99]:
results.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,0.000000,0.000000,0.000000,0.101384,0.898616
1,2,0.537202,0.001899,0.025633,0.261750,0.173516
2,3,0.398684,0.000000,0.020811,0.099136,0.481368
3,4,0.015740,0.015873,0.080131,0.081587,0.806668
4,5,0.443105,0.001524,0.038960,0.312704,0.203707


In [100]:
results.to_csv("submission.csv", index=False)

In [124]:

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [125]:
#logit = LogisticRegression()
#check for over fitting!
#cross_val_score(logit, X_train, y_train, 'accuracy', cv=10)

array([ 0.63224299,  0.63785047,  0.63253857,  0.63002806,  0.61927035,
        0.6323667 ,  0.61880262,  0.62441534,  0.63283442,  0.63529963])

In [139]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

#X_test = kobe_test[feature_columns]
#y_test = kobe_test.shot_made_flag # y_true

#kobe_test['pred'] = model.predict(X_test) # throws an warning / error

#accuracy_score(y_test, kobe_test.pred) # out of sample accuracy

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [140]:
y_pred_val = model.predict(X_val)

In [141]:
y_pred_val

array(['Return_to_owner', 'Adoption', 'Adoption', ..., 'Return_to_owner',
       'Return_to_owner', 'Transfer'], dtype=object)

In [142]:
accuracy_score(y_val, y_pred_val)

0.63150018705574262

In [159]:
model.score(X_val, y_val)

0.63150018705574262

In [146]:
# build a classifier with scikit-learn

import sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

A1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 2),
                        n_estimators = 100,
                        learning_rate = 0.1)

# classifiers = [c.fit(X_train.values[:,1:-1],
#                      X_train.values[:,-1].astype(int)) \
#                for c in [A1]]
# results = [c.predict_proba(test.values[:,1:-1]) \
#            for c in classifiers]
# print(results[0])

A1.fit(X_train, y_train)
real_test_errors =[]
y_pred = A1.predict_proba(test)

In [152]:
y_pred.shape

(11456, 5)

In [157]:
#what is going on here omg?!
accuracy_score(test, y_pred)

ValueError: continuous-multioutput is not supported

In [ ]:
results['Adoption'], results['Died'], results['Euthanasia'], results['Return_to_owner'], results['Transfer'] = y_pred[:,0], y_pred[:,1], y_pred[:,2], y_pred[:,3], y_pred[:,4]
